# Reading and Downloading DEM and LPC Products

The `coincident.io.xarray` and `coincident.io.download` modules support the option to load DEMs into memory via odc-stac or download DEMs into local directories. 

There is also support for Lidar Point Cloud (LPC) spatial filtering for aerial lidar catalogs, where the user can return a GeoDataFrame with the respective .laz tile filename, download url, and geometry (epsg 4326) for each tile intersecting an input aoi. These laz files can then be downloaded locally with `coincident.io.download.download_files()`

There is specific support for USGS 3DEP EPT readers where the user can return a PDAL pipeline configured with the EPT URL, the AOI's bounds, and polygon WKT, all in the EPT's spatial reference system.

```{note}
Coincident *does not support the processing* of lidar point cloud products. Please see the [lidar_tools](https://github.com/uw-cryo/lidar_tools) repository for information on processing the returned GeoDataFrame with lidar point cloud products.
```

In [ ]:
import coincident
import geopandas as gpd
from shapely.geometry import box

## 3DEP and NEON overlapping Flights

```{note}
For all of these functions, you will need identification metadata from the `coincident.search.search` functions for each respective catalog
```

### Search

In [ ]:
workunit = "CO_CentralEasternPlains_1_2020"
df_wesm = coincident.search.wesm.read_wesm_csv()
gf_usgs = coincident.search.wesm.load_by_fid(
    df_wesm[df_wesm.workunit == workunit].index
)

gf_usgs

In [ ]:
# We will examine the 'sourcedem' 1m
# NOTE: it's important to note the *vertical* CRS of the data
gf_usgs.iloc[0]

Now, we'll explore an overlapping NEON flight

In [ ]:
gf_neon = coincident.search.search(
    dataset="neon", intersects=gf_usgs, datetime=["2020"]
)

In [ ]:
gf_neon.head()

In [ ]:
m = gf_usgs.explore(color="black")
gf_neon.explore(m=m, column="id", popups=True)

### Subset data

We will evaluate small subset of this overlap for deomstrative purposes.

Let's subset based on some contextual LULC data

In [ ]:
gf_wc = coincident.search.search(
    dataset="worldcover",
    intersects=gf_neon,
    datetime=["2020"],
)

In [ ]:
dswc = coincident.io.xarray.to_dataset(
    gf_wc,
    bands=["map"],
    aoi=gf_neon,
    mask=True,
    resolution=0.00027,  # ~30m
)
dswc = dswc.rename(map="landcover")
dswc = dswc.compute()

In [ ]:
# arbitrary bbox that will be our subset area (all cropland)
bbox_geometry = box(-102.505, 39.675, -102.49, 39.685)
aoi = gpd.GeoDataFrame(geometry=[bbox_geometry], crs="EPSG:4326")

In [ ]:
ax = coincident.plot.plot_esa_worldcover(dswc)
aoi.plot(ax=ax, facecolor="none", edgecolor="black", linestyle="--", linewidth=2)
from matplotlib.lines import Line2D

custom_line = Line2D([0], [0], color="black", linestyle="--", lw=2)
ax.legend([custom_line], ["Area of Interest"], loc="upper right", fontsize=10)
ax.set_title("ESA WorldCover");

Actually read in the DEMs

In [ ]:
datetime_str = gf_neon.end_datetime.item()
site_id = gf_neon.id.item()
datetime_str, site_id

In [ ]:
%%time
da_neon_dem = coincident.io.xarray.load_neon_dem(
    aoi, datetime_str=datetime_str, site_id=site_id, product="dsm"
)

In [ ]:
da_neon_dem

In [ ]:
usgs_project = gf_usgs["project"].item()
usgs_project

In [ ]:
%%time
da_usgs_dem = coincident.io.xarray.load_usgs_dem(aoi, usgs_project)

In [ ]:
da_usgs_dem

In [ ]:
da_usgs_dem.coarsen(x=5, y=5, boundary="trim").mean().plot.imshow();

### Download

```{note}
`coincident.io.download.download_neon_dem` needs the NEON site's start_datetime OR end_datetime to work
```

In [ ]:
gf_neon

In [ ]:
local_output_dir = "/tmp"

In [ ]:
coincident.io.download.download_neon_dem(
    aoi=aoi,
    datetime_str=gf_neon.start_datetime.item(),
    site_id=gf_neon.id.item(),
    product="dsm",
    output_dir=local_output_dir,
)

In [ ]:
# USGS_1M_13_x71y440_CO_CentralEasternPlains_2020_D20.tif:  236MB
coincident.io.download.download_usgs_dem(
    aoi=aoi,
    project=usgs_project,
    output_dir=local_output_dir,
    save_parquet=True,  # save a STAC-like geoparquet of the tiles you download
)

Finally, you can grab the LPC tile metadata. For USGS 3DEP data, you can also return a PDAL pipeline based on the available EPT data. This PDAL pipeline will be returned as a JSON file where the user can add their own custom parameters (additional filters, writers, etc.) to this pipeline dictionary before executing it with PDAL.

```{note}
`coincident.io.download.fetch_lpc_tiles` needs the NEON site's end_datetime OR start_datetime to work if you are in fact accessing NEON data
```

In [ ]:
%%time
gf_neon_lpc_tiles = coincident.io.download.fetch_lpc_tiles(
    aoi=aoi,
    dataset_id=gf_neon.id.item(),
    provider="NEON",
    datetime_str=gf_neon.end_datetime.item(),
)

In [ ]:
gf_neon_lpc_tiles.head()

In [ ]:
%%time
gf_usgs_lpc_tiles = coincident.io.download.fetch_lpc_tiles(
    aoi=aoi, dataset_id=usgs_project, provider="USGS"
)

In [ ]:
gf_usgs_lpc_tiles.head()

In [ ]:
m = gf_usgs_lpc_tiles.explore(color="black")
gf_neon_lpc_tiles.explore(m=m)

Now, we can download the laz files

In [ ]:
coincident.io.download.download_files(
    gf_neon_lpc_tiles["url"], output_dir=local_output_dir
)

In [ ]:
pdal_pipeline = coincident.io.download.build_usgs_ept_pipeline(
    aoi, workunit=gf_usgs.workunit.item(), output_dir=local_output_dir
)

In [ ]:
pdal_pipeline

## NCALM

### Search

In [ ]:
aoi = gpd.read_file(
    "https://raw.githubusercontent.com/unitedstates/districts/refs/heads/gh-pages/states/WA/shape.geojson"
)
aoi.plot();

In [ ]:
gf_ncalm = coincident.search.search(
    dataset="ncalm", intersects=aoi, datetime=["2018-09-19"]
)

In [ ]:
gf_ncalm

Now, let's subset to a small AOI for convenience sake

In [ ]:
buffer_size = 0.01
centroid = gf_ncalm.centroid
mini_aoi = gpd.GeoDataFrame(
    geometry=[
        box(
            centroid.x - buffer_size,
            centroid.y - buffer_size,
            centroid.x + buffer_size,
            centroid.y + buffer_size,
        )
    ],
    crs="EPSG:4326",
)

In [ ]:
m = gf_ncalm.explore()
mini_aoi.explore(m=m, color="red")

```{note}
The NCALM DEMs are not tiled, so reading in the data and downloading takes a longer time
```

### Subset data

In [ ]:
%%time
da_ncalm_dtm = coincident.io.xarray.load_ncalm_dem(
    aoi=mini_aoi, product="dtm", dataset_id=gf_ncalm["name"].item()
)

In [ ]:
da_ncalm_dtm

In [ ]:
da_ncalm_dtm.coarsen(x=5, y=5, boundary="trim").mean().plot.imshow();

### Download

```{note}
You will not see a download progress bar for downloading NCALM DEMs as opposed to USGS and NEON provider DEMs
```

In [ ]:
%%time
coincident.io.download.download_ncalm_dem(
    aoi=mini_aoi,
    dataset_id=gf_ncalm["name"].item(),
    product="dtm",
    output_dir=local_output_dir,
)

In [ ]:
%%time
gf_ncalm_lpc_tiles = coincident.io.download.fetch_lpc_tiles(
    aoi=mini_aoi, dataset_id=gf_ncalm.name.item(), provider="NCALM"
)

In [ ]:
gf_ncalm_lpc_tiles.head()

In [ ]:
m = gf_ncalm.explore()
gf_ncalm_lpc_tiles.explore(m=m, color="red")
mini_aoi.explore(m=m, color="black")

## NOAA

### Search

In [ ]:
aoi = gpd.read_file(
    "https://raw.githubusercontent.com/unitedstates/districts/refs/heads/gh-pages/states/FL/shape.geojson"
)
gf_noaa = coincident.search.search(
    dataset="noaa", intersects=aoi, datetime=["2022-10-27"]
)

In [ ]:
buffer_size = 0.02
centroid = gf_noaa.centroid
mini_aoi = gpd.GeoDataFrame(
    geometry=[
        box(
            centroid.x - buffer_size,
            centroid.y - buffer_size,
            centroid.x + buffer_size,
            centroid.y + buffer_size,
        )
    ],
    crs="EPSG:4326",
)

In [ ]:
m = gf_noaa.explore()
mini_aoi.explore(m=m, color="red")

the name and id being identical is expected

In [ ]:
gf_noaa

```{note}
Our coincident.search.search(dataset="noaa") returns the dataset ids "Lidar Datasets at NOAA Digital Coast" whereas coincident.io.xarray.load_noaa_dem() requires the ids from the "Imagery and Elevation Raster Datasets at NOAA Digital Coast" dataset. The corresponding elevation raster dataset id is the same as the lidar dataset id + 1. 
e.g. "Great Bay NERR UAS Lidar" has id 10175 for lidar data and id 10176 for dem data
```

In [ ]:
noaa_dem_id = int(gf_noaa.id.item()) + 1
print(f"NOAA LiDAR id: {gf_noaa.id.item()}  NOAA DEM id: {noaa_dem_id}")

```{warning}
The larger the NOAA flight, the longer the below function takes regardless of your input AOI
```

### Subset

In [ ]:
%%time
da_noaa_dem = coincident.io.xarray.load_noaa_dem(mini_aoi, noaa_dem_id)

In [ ]:
da_noaa_dem

In [ ]:
da_noaa_dem.coarsen(x=50, y=50, boundary="trim").mean().plot.imshow();

In [ ]:
buffer_size = 0.008
centroid = gf_noaa.centroid
mini_aoi = gpd.GeoDataFrame(
    geometry=[
        box(
            centroid.x - buffer_size,
            centroid.y - buffer_size,
            centroid.x + buffer_size,
            centroid.y + buffer_size,
        )
    ],
    crs="EPSG:4326",
)
m = gf_noaa.explore()
mini_aoi.explore(m=m, color="red")

### Download

```{note}
You will not see a download progress bar for downloading NOAA DEMs as opposed to USGS and NEON provider DEMs
```

In [ ]:
%%time
coincident.io.download.download_noaa_dem(
    aoi=mini_aoi, dataset_id=noaa_dem_id, output_dir=local_output_dir
)

In [ ]:
list(mini_aoi.bounds.values)

In [ ]:
gf_noaa.id.item()

In [ ]:
%%time
gf_noaa_lpc_tiles = coincident.io.download.fetch_lpc_tiles(
    aoi=mini_aoi, dataset_id=gf_noaa.id.item(), provider="NOAA"
)

In [ ]:
gf_noaa_lpc_tiles

In [ ]:
m = gf_noaa.explore()
gf_noaa_lpc_tiles.explore(m=m, color="red")

## G-LiHT

### Search

In [ ]:
aoi = gpd.read_file(
    "https://raw.githubusercontent.com/unitedstates/districts/refs/heads/gh-pages/states/MD/shape.geojson"
)
aoi = aoi.simplify(0.01)
aoi.explore()

In [ ]:
gf_gliht = coincident.search.search(
    dataset="gliht",
    intersects=aoi,
    datetime=["2017-07-31"],
)

In [ ]:
gf_gliht = gf_gliht.iloc[[0]]

In [ ]:
gf_gliht

Now, let's create a small areal subset of the data as we have for the other examples

In [ ]:
c = gf_gliht.geometry.centroid
mini_aoi = gpd.GeoDataFrame(
    geometry=[box(c.x - 0.0045, c.y - 0.0045, c.x + 0.0045, c.y + 0.0045)],
    crs="EPSG:4326",
)

In [ ]:
m = gf_gliht.explore()
mini_aoi.explore(color="black", m=m)

As seen in the `Additional Aerial LiDAR Datasets` example, NASA G-LiHT has many different provided gridded datasets. The following collections below are the current datasets supported by `coincident`.

```python
    - 'ortho': Orthorectified aerial imagery
    - 'chm': Canopy height model
    - 'dsm': Digital surface model
    - 'dtm': Digital terrain model
    - 'hyperspectral_ancillary': Ancillary HSI data
    - 'radiance': Hyperspectral aradiance
    - 'reflectance': Hyperspectral surface reflectance
    - 'hyperspectral_vegetation': HSI-derived veg indices
```

```{note}
Not all G-LiHT flights will contain every single product listed. For example, a flight may have 'dsm' data but not 'ortho' data.
```

In [ ]:
# we'll need the dataset id from our initial search
gf_gliht.id.item()

```{important}
Unlike the G-LiHT search, you will need NASA Earthdata credentials to read in and download the gridded datasets from G-LiHT. You can register for a free account here:
https://urs.earthdata.nasa.gov/users/new

and set:
```

```{python}
import os

os.environ["EARTHDATA_USER"] = "username"
os.environ["EARTHDATA_PASS"] = "password"
```


In [ ]:
%%time
da_chm = coincident.io.xarray.load_gliht_raster(
    aoi=mini_aoi,
    dataset_id=gf_gliht.id.item(),
    product="chm",
)

In [ ]:
da_chm

In [ ]:
da_chm.plot.imshow(cmap="Greens");

### Download

```{note}
Since there are so many gridded products for G-LiHT, you can specify multiple products to download in one call. This download will automatically skip any products that aren't available for the specific flight among the current datasets supported by `coincident`
```

In [ ]:
coincident.io.download.download_gliht_raster(
    aoi=mini_aoi,
    dataset_id=gf_gliht.id.item(),
    products=["chm", "dsm", "dtm"],
    output_dir="/tmp",
)